In [1]:
from pyspark.sql import functions as F 
from pyspark.sql.window import Window
from pyspark.sql.types import *
from pyspark.storagelevel import StorageLevel

In [2]:
import pyspark.sql.functions as F
data = [
    ('a', 'x1'),
    ('a', 'x2'),
    ('a', 'x3'),
    ('b', 'y1'),
    ('b', 'y2')
]
df = spark.createDataFrame(data, ['name', 'val'])

In [3]:
df.withColumn("uuid", F.when(F.col("name")=='a', F.monotonically_increasing_id()).otherwise(F.lit(0))).show()

+----+---+-----------+
name|val| uuid|
+----+---+-----------+
 a| x1| 8589934592|
 a| x2|25769803776|
 a| x3|34359738368|
 b| y1| 0|
 b| y2| 0|
+----+---+-----------+

In [4]:
result=(df.
        groupby('id').
        agg(
            F.collect_set(F.col('val')).alias('arr_of_vals')
        ).
        withColumn(
            'arr_to_string',
            F.concat_ws(
                ',',
                'arr_of_vals')
            )
        )

In [5]:
result.show()

+---+------------+-------------+
 id| arr_of_vals|arr_to_string|
+---+------------+-------------+
 b| [y2, y1]| y2,y1|
 a|[x1, x3, x2]| x1,x3,x2|
+---+------------+-------------+

In [6]:
list=[["car",["url1","url10"],"url1"],
["monkey",["url11","url20"],"url11"],
["mouse",["url21","url30"],"url11"],
["donkey",["url31","url40"],""""""""],
["ball",["url41","url45","url50"],"url45"],
["monkey",["url11","url20"],"url1"]]

df=spark.createDataFrame(list,['query','hrefs','clicked_url'])

df.show()

+------+--------------------+-----------+
 query| hrefs|clicked_url|
+------+--------------------+-----------+
 car| [url1, url10]| url1|
monkey| [url11, url20]| url11|
 mouse| [url21, url30]| url11|
donkey| [url31, url40]| |
 ball|[url41, url45, ur...| url45|
monkey| [url11, url20]| url1|
+------+--------------------+-----------+

In [7]:
from pyspark.sql import functions as F
df.withColumn("clicked_url", F.when(F.col("clicked_url")=="", F.lit(0))\
              .otherwise(F.col("clicked_url")))\
  .filter("array_contains(hrefs,clicked_url)=true or clicked_url=0").show()

+------+--------------------+-----------+
 query| hrefs|clicked_url|
+------+--------------------+-----------+
 car| [url1, url10]| url1|
monkey| [url11, url20]| url11|
donkey| [url31, url40]| 0|
 ball|[url41, url45, ur...| url45|
+------+--------------------+-----------+

In [8]:

list=[['a1', 12016668],
['a2', 22653585],
['a3',107313117],
['a4',       69],
['a5',        5],
['a6', 13092243],
['a7',      372],
['a8',     3167],
['a9', 18712794],
['a10',  2456488],
['a11',  2733665],
['a12', 14854475]]

df=spark.createDataFrame(list,['var1','Total'])
df.show()


+----+---------+
var1| Total|
+----+---------+
 a1| 12016668|
 a2| 22653585|
 a3|107313117|
 a4| 69|
 a5| 5|
 a6| 13092243|
 a7| 372|
 a8| 3167|
 a9| 18712794|
 a10| 2456488|
 a11| 2733665|
 a12| 14854475|
+----+---------+

In [9]:
df.agg(F.sum("Total")).collect()[0][0]

Out[40]: 193836648

In [10]:
df.withColumn("porportion", (F.col("Total")/F.lit(df.agg(F.sum("Total")).collect()[0][0]))).show()

+----+---------+--------------------+
var1| Total| porportion|
+----+---------+--------------------+
 a1| 12016668| 0.06199378767631186|
 a2| 22653585| 0.1168694631987239|
 a3|107313117| 0.5536265618873063|
 a4| 69|3.559698370351513...|
 a5| 5|2.579491572718488...|
 a6| 13092243| 0.06754266097296524|
 a7| 372|1.919141730102555...|
 a8| 3167|1.633849962159890...|
 a9| 18712794| 0.09653898885003417|
 a10| 2456488|0.012672980188968187|
 a11| 2733665|0.014102931660270972|
 a12| 14854475| 0.07663398615931492|
+----+---------+--------------------+

In [11]:
N = df.agg(F.sum(F.col('Total'))).collect()[0][0]


df.withColumn('Proportion',  F.format_number(df.Total / N, 4)).show()

+----+---------+----------+
var1| Total|Proportion|
+----+---------+----------+
 a1| 12016668| 0.0620|
 a2| 22653585| 0.1169|
 a3|107313117| 0.5536|
 a4| 69| 0.0000|
 a5| 5| 0.0000|
 a6| 13092243| 0.0675|
 a7| 372| 0.0000|
 a8| 3167| 0.0000|
 a9| 18712794| 0.0965|
 a10| 2456488| 0.0127|
 a11| 2733665| 0.0141|
 a12| 14854475| 0.0766|
+----+---------+----------+

In [12]:
list=[['[{"name":"name1","id":"1"}, {"name":"name2","id":"2"}]']]
df=spark.createDataFrame(list,['mycol'])
df.show(truncate=False)
df.printSchema()
json_schema=ArrayType(StructType([ StructField("name", StringType()), StructField("id", StringType())]))

df1=df.withColumn("json",(F.from_json("mycol",json_schema))).select("json")

+------------------------------------------------------+
mycol |
+------------------------------------------------------+
[{"name":"name1","id":"1"}, {"name":"name2","id":"2"}]|
+------------------------------------------------------+

root
-- mycol: string (nullable = true)

In [13]:
df1.printSchema()

root
-- json: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- name: string (nullable = true)
 | |-- id: string (nullable = true)

In [14]:
df1.selectExpr("flatten(json)").withColumn.show(truncate=False)

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 

 Py4JJavaError : An error occurred while calling o912.selectExpr.
: org.apache.spark.sql.AnalysisException: cannot resolve 'flatten(`json`)' due to data type mismatch: The argument should be an array of arrays, but '`json`' is of array<struct<name:string,id:string>> type.; line 1 pos 0;
'Project [unresolvedalias(flatten(json#2924), Some(<function1>))]
+- Project [json#2924]
 +- Project [mycol#2918, jsontostructs(ArrayType(StructType(StructField(name,StringType,true), StructField(id,StringType,true)),true), mycol#2918, Some(Etc/UTC)) AS json#2924]
 +- LogicalRDD [mycol#2918], false

	at org.apache.spark.sql.catalyst.analysis.package$AnalysisErrorAt.failAnalysis(package.scala:42)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis$$anonfun$checkAnalysis$1$$anonfun$apply$3.applyOrElse(CheckAnalysis.scala:126)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis$$anonfun$checkAnalysis$1$$anonfun$apply$3.applyOrElse(CheckAnalysis.scala:111)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$6.apply(TreeNode.scala:303)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$6.apply(TreeNode.scala:303)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(TreeNode.scala:76)
	at org.apache.spark.sql.catalyst.trees.TreeNode.transformUp(TreeNode.scala:302)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$5.apply(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$5.apply(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$8.apply(TreeNode.scala:353)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapProductIterator(TreeNode.scala:207)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapChildren(TreeNode.scala:351)
	at org.apache.spark.sql.catalyst.trees.TreeNode.transformUp(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$transformExpressionsUp$1.apply(QueryPlan.scala:94)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$transformExpressionsUp$1.apply(QueryPlan.scala:94)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$3.apply(QueryPlan.scala:106)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$3.apply(QueryPlan.scala:106)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(TreeNode.scala:76)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.transformExpression$1(QueryPlan.scala:105)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.org$apache$spark$sql$catalyst$plans$QueryPlan$$recursiveTransform$1(QueryPlan.scala:117)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$org$apache$spark$sql$catalyst$plans$QueryPlan$$recursiveTransform$1$2.apply(QueryPlan.scala:122)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collection.mutable.ResizableArray$class.foreach(ResizableArray.scala:59)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:48)
	at scala.collection.TraversableLike$class.map(TraversableLike.scala:234)
	at scala.collection.AbstractTraversable.map(Traversable.scala:104)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.org$apache$spark$sql$catalyst$plans$QueryPlan$$recursiveTransform$1(QueryPlan.scala:122)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$4.apply(QueryPlan.scala:127)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapProductIterator(TreeNode.scala:207)
	at org.a

In [15]:
list=[['[{"name":"name1","id":"1"}, {"name":"name2","id":"2"}]']]
#schema= StructType([MapType(StructField("name",StringType(),StringType()),
                             #StructField("id", StringType(), StringType()))])

df=spark.createDataFrame(list,['json'])
from pyspark.sql.functions import from_json, col
df.printSchema()
#json_schema = spark.read.json(df.rdd.map(lambda row: row.json)).schema
json_schema=ArrayType(StructType([ StructField("name", StringType()), StructField("id", StringType())]))
#json_schema1=StructType([(StructField(id,StringType(),True),StructField(name,StringType(),True))])
df.withColumn("yo",F.explode(F.from_json("json",json_schema))).select("yo.*").show()

root
-- json: string (nullable = true)

+-----+---+
 name| id|
+-----+---+
name1| 1|
name2| 2|
+-----+---+

In [16]:
d = [(100,1,23,10),(100,2,45,11),(100,3,67,12),(100,4,78,13),(101,1,23,10),(101,2,45,13),(101,3,67,14),(101,4,78,15),(102,1,23,10),(102,2,45,11),(102,3,67,16),(102,4,78,18)]
mydf = spark.createDataFrame(d,['id','day','price','units'])
mydf.show()

+---+---+-----+-----+
 id|day|price|units|
+---+---+-----+-----+
100| 1| 23| 10|
100| 2| 45| 11|
100| 3| 67| 12|
100| 4| 78| 13|
101| 1| 23| 10|
101| 2| 45| 13|
101| 3| 67| 14|
101| 4| 78| 15|
102| 1| 23| 10|
102| 2| 45| 11|
102| 3| 67| 16|
102| 4| 78| 18|
+---+---+-----+-----+

In [17]:
mydf.groupby('id').pivot('day').agg(F.mean('units'),F.mean('price')).show()

+---+------------+------------+------------+------------+------------+------------+------------+------------+
 id|1_avg(units)|1_avg(price)|2_avg(units)|2_avg(price)|3_avg(units)|3_avg(price)|4_avg(units)|4_avg(price)|
+---+------------+------------+------------+------------+------------+------------+------------+------------+
100| 10.0| 23.0| 11.0| 45.0| 12.0| 67.0| 13.0| 78.0|
101| 10.0| 23.0| 13.0| 45.0| 14.0| 67.0| 15.0| 78.0|
102| 10.0| 23.0| 11.0| 45.0| 16.0| 67.0| 18.0| 78.0|
+---+------------+------------+------------+------------+------------+------------+------------+------------+

In [18]:
from pyspark.sql import functions as F 
from pyspark.sql.window import Window
from pyspark.sql.types import *
from pyspark.storagelevel import StorageLevel

In [19]:
list=[[1,'2019-01-01','2019-03-01',30],
      [2,'2019-01-01','2019-06-01',20],
      [3,'2019-07-01','2020-08-01',10],
      [4,'2019-07-01','2020-08-01',10]]
df=spark.createDataFrame(list,['id','start_period','end_period','montly_price'])
df.show()

+---+------------+----------+------------+
 id|start_period|end_period|montly_price|
+---+------------+----------+------------+
 1| 2019-01-01|2019-03-01| 30|
 2| 2019-01-01|2019-06-01| 20|
 3| 2019-07-01|2020-08-01| 10|
 4| 2019-07-01|2020-08-01| 10|
+---+------------+----------+------------+

In [21]:
import datetime
def generate_dates(start, stop):
    return [start + datetime.timedelta(weeks=x) for x in range(0, (stop-start).weeks + 1)]
  
udf1=F.udf(generate_dates, ArrayType(DateType()))

df.withColumn("period", F.explode(udf1(F.to_date("start_period"),F.to_date("end_period"))))\
  .withColumn("Month", F.month("period"))\
  .withColumn("Year", F.year("period"))\
  .withColumn("montly_price", F.when(F.col("end_period")==F.col("period"),F.lit(0)).otherwise(F.col("montly_price")))\
  .withColumn("period", F.date_format("period","yyyy-MM"))\
  .groupBy("year","month").agg(F.first("period").alias("period"), F.sum("montly_price").alias("revenue_per_month_all_ids"))\
  .drop("year","month")\
  .orderBy("period").show()


--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-3721676703577033> in <module> 
 12 . groupBy ( "year" , "month" ) . agg ( F . first ( "period" ) . alias ( "period" ) , F . sum ( "montly_price" ) . alias ( "revenue_per_month_all_ids" ) ) \ 
 13 . drop ( "year" , "month" ) \ 
 ---> 14 . orderBy ( "period" ) . show ( ) 

 /databricks/spark/python/pyspark/sql/dataframe.py in show (self, n, truncate, vertical) 
 382 """
 383 if isinstance ( truncate , bool ) and truncate : 
 --> 384 print ( self . _jdf . showString ( n , 20 , vertical ) ) 
 385 else : 
 386 print ( self . _jdf . showString ( n , int ( truncate ) , vertical ) ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1255 answer = self . gateway_client . send_command ( command ) 
 1256 return_value = get_return_value(
 -> 1257 answer, self.gateway_client, self.target_id, self.name)
 1258 
 1259 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 61 def deco ( * a , ** kw ) : 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 
 65 s = e . java_exception . toString ( ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 
 330 raise Py4JError(

 Py4JJavaError : An error occurred while calling o1179.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 17.0 failed 1 times, most recent failure: Lost task 1.0 in stage 17.0 (TID 259, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
 File "/databricks/spark/python/pyspark/worker.py", line 480, in main
 process()
 File "/databricks/spark/python/pyspark/worker.py", line 472, in process
 serializer.dump_stream(out_iter, outfile)
 File "/databricks/spark/python/pyspark/serializers.py", line 460, in dump_stream
 self.serializer.dump_stream(self._batched(iterator), stream)
 File "/databricks/spark/python/pyspark/serializers.py", line 150, in dump_stream
 for obj in iterator:
 File "/databricks/spark/python/pyspark/serializers.py", line 449, in _batched
 for item in iterator:
 File "<string>", line 1, in <lambda>
 File "/databricks/spark/python/pyspark/worker.py", line 85, in <lambda>
 return lambda *a: toInternal(f(*a))
 File "/databricks/spark/python/pyspark/util.py", line 99, in wrapper
 return f(*args, **kwargs)
 File "<command-3721676703577033>", line 3, in generate_dates
AttributeError: 'datetime.timedelta' object has no attribute 'weeks'

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:540)
	at org.apache.spark.sql.execution.python.PythonUDFRunner$$anon$1.read(PythonUDFRunner.scala:81)
	at org.apache.spark.sql.execution.python.PythonUDFRunner$$anon$1.read(PythonUDFRunner.scala:64)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:494)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIterator.scala:37)
	at scala.collection.Iterator$$anon$12.hasNext(Iterator.scala:440)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage2.sort_addToSorter_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage2.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$13$$anon$1.hasNext(WholeStageCodegenExec.scala:640)
	at org.apache.spark.sql

In [22]:
import datetime
def generate_dates(start, stop):
    return [start + datetime.timedelta(months=x) for x in range(0, (stop-start).months + 1)]
  
udf1=F.udf(generate_dates, ArrayType(DateType()))

df.withColumn("period", F.explode(udf1(F.to_date("start_period"),F.to_date("end_period")))).show()

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-3721676703577034> in <module> 
 5 udf1 = F . udf ( generate_dates , ArrayType ( DateType ( ) ) ) 
 6 
 ----> 7 df . withColumn ( "period" , F . explode ( udf1 ( F . to_date ( "start_period" ) , F . to_date ( "end_period" ) ) ) ) . show ( ) 

 /databricks/spark/python/pyspark/sql/dataframe.py in show (self, n, truncate, vertical) 
 382 """
 383 if isinstance ( truncate , bool ) and truncate : 
 --> 384 print ( self . _jdf . showString ( n , 20 , vertical ) ) 
 385 else : 
 386 print ( self . _jdf . showString ( n , int ( truncate ) , vertical ) ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1255 answer = self . gateway_client . send_command ( command ) 
 1256 return_value = get_return_value(
 -> 1257 answer, self.gateway_client, self.target_id, self.name)
 1258 
 1259 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 61 def deco ( * a , ** kw ) : 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 
 65 s = e . java_exception . toString ( ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 
 330 raise Py4JError(

 Py4JJavaError : An error occurred while calling o608.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 2 in stage 10.0 failed 1 times, most recent failure: Lost task 2.0 in stage 10.0 (TID 232, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
 File "/databricks/spark/python/pyspark/worker.py", line 480, in main
 process()
 File "/databricks/spark/python/pyspark/worker.py", line 472, in process
 serializer.dump_stream(out_iter, outfile)
 File "/databricks/spark/python/pyspark/serializers.py", line 460, in dump_stream
 self.serializer.dump_stream(self._batched(iterator), stream)
 File "/databricks/spark/python/pyspark/serializers.py", line 150, in dump_stream
 for obj in iterator:
 File "/databricks/spark/python/pyspark/serializers.py", line 449, in _batched
 for item in iterator:
 File "<string>", line 1, in <lambda>
 File "/databricks/spark/python/pyspark/worker.py", line 85, in <lambda>
 return lambda *a: toInternal(f(*a))
 File "/databricks/spark/python/pyspark/util.py", line 99, in wrapper
 return f(*args, **kwargs)
 File "<command-3721676703577034>", line 3, in generate_dates
AttributeError: 'datetime.timedelta' object has no attribute 'months'

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:540)
	at org.apache.spark.sql.execution.python.PythonUDFRunner$$anon$1.read(PythonUDFRunner.scala:81)
	at org.apache.spark.sql.execution.python.PythonUDFRunner$$anon$1.read(PythonUDFRunner.scala:64)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:494)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIterator.scala:37)
	at scala.collection.Iterator$$anon$12.hasNext(Iterator.scala:440)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage2.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$13$$anon$1.hasNext(WholeStageCodegenExec.scala:640)
	at org.apache.spark.sql.execution.collect.UnsafeRowBatchUtils$.encodeUnsafeRows(UnsafeRowBatchUtils.scala:62)
	at org.apache.spark.sql.execution.collect.Collector$$anonfun$2.apply(Coll

In [23]:
df.withColumn("period", F.explode(F.expr("""sequence(to_date(start_period),to_date(end_period), interval 1 month)""")))\
  .withColumn("Month", F.month("period"))\
  .withColumn("Year", F.year("period"))\
  .withColumn("montly_price", F.when(F.col("end_period")==F.col("period"),F.lit(0)).otherwise(F.col("montly_price")))\
  .withColumn("period", F.date_format("period","yyyy-MM"))\
  .groupBy("year","month").agg(F.first("period").alias("period"), F.sum("montly_price").alias("revenue_per_month_all_ids"))\
  .drop("year","month")\
  .orderBy("period").show()

+-------+-------------------------+
 period|revenue_per_month_all_ids|
+-------+-------------------------+
2019-01| 50|
2019-02| 50|
2019-03| 20|
2019-04| 20|
2019-05| 20|
2019-06| 0|
2019-07| 20|
2019-08| 20|
2019-09| 20|
2019-10| 20|
2019-11| 20|
2019-12| 20|
2020-01| 20|
2020-02| 20|
2020-03| 20|
2020-04| 20|
2020-05| 20|
2020-06| 20|
2020-07| 20|
2020-08| 0|
+-------+-------------------------+

In [24]:
period          revenue_per_month_all_ids
2019-01         50
2019-02         50
2019-04         20
2019-05         20
2019-06         0
2019-07         20